### Loading a file from GCS bucket into BQ using Google APIs
Below is some sample code w/documentation that is runnable from within this notebook on loading a file from a gcs bucket into bq WITHOUT using pandas. <br>
Feel free to run this as manytimes as you would like everything is done in lab-e00.  Also, below are some good links with examples.

> Links: 
- GCP-Python Client Library: https://google-cloud.readthedocs.io/en/latest/index.html   ***Highly Recommended
- Introduction to loading data into BQ: https://cloud.google.com/bigquery/docs/loading-data
- GCP Authentication: https://google-auth.readthedocs.io/en/latest/index.html
- BQ Batch/Interactive Queries: https://cloud.google.com/bigquery/docs/running-queries


In [1]:
# Imports
from google.cloud import bigquery
from google.oauth2 import service_account

# Define project parms; datasets, tables, etc
stg_project = 'lab-e00-ent'
stg_dataset_id = 'temporary'
tgt_dataset_id = 'discovery'
stg_table = 'example_stg_01'
tgt_table = 'example_tgt_01'
cred_file = '/data/keys/lab-e00.json'
gcs_bucket = 'fd_testing'
filename = 'PandC_RPM_Mailed_Triggers.csv'


def load_file_to_bq():

    # Instantiate a credential object from serevice accoutn JSON file
    # You may also see 'from_service_account_json'.. file is what I have seen mainly used
    # https://google-auth.readthedocs.io/en/latest/_modules/google/oauth2/service_account.html
    stg_credentials = service_account.Credentials.from_service_account_file(cred_file)

    # Instantiate a BQ Client from project and credentials
    # Here we are calling the client stg_client becuase we may create another client for a different project. eg. moving data from LAB to VALUE
    stg_client = bigquery.Client(project=stg_project, credentials=stg_credentials)

    # Create a dataset/reference object.  Will used this instantiated object to set/create table name in the desired dataset
    dataset_ref = stg_client.dataset(stg_dataset_id)

    # Get defined BQ job configuration   https://google-cloud.readthedocs.io/en/latest/bigquery/generated/google.cloud.bigquery.job.LoadJobConfig.html
    # This will load default config and allow us to define any parmaters for this load
    job_config = bigquery.LoadJobConfig()

    # Set job config to truncate the current table if it exists
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

    # Define the schema of the BQ table
    job_config.schema = [
        bigquery.SchemaField("PM_FIRST_NAME", "STRING"),
        bigquery.SchemaField("RPM_MIDDLE_INITIAL", "STRING"),
        bigquery.SchemaField("RPM_LAST_NAME", "STRING"),
        bigquery.SchemaField("RPM_NAME_SUFFIX", "STRING"),
        bigquery.SchemaField("RPM_ADDRESS_LINE1", "STRING"),
        bigquery.SchemaField("RPM_ADDRESS_LINE2", "STRING"),
        bigquery.SchemaField("RPM_CITY", "STRING"),
        bigquery.SchemaField("RPM_STATE", "STRING"),
        bigquery.SchemaField("RPM_ZIP", "STRING"),
        bigquery.SchemaField("RPM_ZIP4", "STRING"),
        bigquery.SchemaField("FILLER1", "STRING"),
        bigquery.SchemaField("CAMPAIGN_CODE", "STRING"),
        bigquery.SchemaField("MODEL_1_DECILE", "STRING"),
        bigquery.SchemaField("CAMPAIGN_SOURCE_CD", "STRING"),
        bigquery.SchemaField("CAMPAIGN_SEQUENCE_NUMBER", "STRING"),
        bigquery.SchemaField("CAMPAIGN_CELL_CD", "STRING"),
        bigquery.SchemaField("CAMPAIGN_MAILDATE", "STRING"),
        bigquery.SchemaField("ASSIST_SCORE_CD", "STRING"),
        bigquery.SchemaField("DATA_SOURCE_INDICATOR", "STRING"),
        bigquery.SchemaField("POC_CODE", "STRING"),
        bigquery.SchemaField("PROMOTION_GROUP_ID", "STRING"),
        bigquery.SchemaField("CREATIVE", "STRING"),
        bigquery.SchemaField("TARGET_TYPE_CD", "STRING"),
        bigquery.SchemaField("COST", "STRING"),
        bigquery.SchemaField("RPM_IID", "STRING"),
        bigquery.SchemaField("FILLER2", "STRING"),
        bigquery.SchemaField("FILLER3", "STRING"),
        bigquery.SchemaField("add_date", "STRING"),
        bigquery.SchemaField("MEMBER_TENURE", "STRING"),
        bigquery.SchemaField("MULTI_DRIVER", "STRING"),
        bigquery.SchemaField("GOOD_STUDENT", "STRING"),
        bigquery.SchemaField("MPD_AUTO_FL", "STRING"),
        bigquery.SchemaField("MPD_AUTO_GA_TN", "STRING")
    ]

    # job_config.source_format = bigquery.SourceFormat.CSV    (This is optional as it is the default)

    # Define the uri of the bucket and filename
    uri = "gs://{}/{}".format(gcs_bucket,filename)  # "gs://fd_testing/PandC_RPM_Mailed_Triggers.csv"

    # Instantiate based on our modified configuration and provide the destination table name
    load_job = stg_client.load_table_from_uri(
        uri, dataset_ref.table(stg_table), job_config=job_config
    )  # API request

    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    # Get results and display.. This is a bit misleading as it will count all rows in the table and not just what you are loading (appends)
    destination_table = stg_client.get_table(dataset_ref.table(stg_table))
    print("Loaded {} rows.".format(destination_table.num_rows))


if __name__ == "__main__":

    try:
        load_file_to_bq()
    except Exception as e:
        s = str(e)
        print("ERROR: " + s)
        raise RuntimeError(e)


Starting job 15d7fef6-e551-4e83-a0e7-338eebdd491e
Job finished.
Loaded 14023202 rows.
